# **(b)**

此部分為最後一天下午才發現忽略了MAP也要做N fold的部分，在此補上。

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics

In [4]:
x = pd.read_csv('data_X.csv').drop('Serial_id',axis=1)
y = pd.read_csv('data_T.csv').drop('Serial_id',axis=1)
x = (x-x.mean())/(x.std())
x = pd.concat([x,y],axis=1)
x = x.reindex(np.random.permutation(x.index))

In [15]:
x_train = x.iloc[:,[0,1,4,5,6]]
y_train = pd.DataFrame(x.iloc[:,-1])

In [17]:
def sigmoid_basisfunc(data):
    new_data = pd.DataFrame()
    for i in range(data.shape[1]):
        temp = pd.DataFrame(data.iloc[:,i]).applymap(lambda x:1/(1+np.exp(-x))) #輸入的data要先做好標準化才行
        new_data = pd.concat([new_data,temp],axis = 1) 
    return new_data #回傳已經代入sigmoid bisis的input

In [18]:
def Weight(x,y,Lambda=0):
    x = pd.concat([pd.Series(1,index=x.index,name = 'w0'),x],axis=1) #將輸入資料補上bias項
    x = x.to_numpy() #轉換成矩陣來運算
    temp = np.linalg.inv(np.eye(x.shape[1])*Lambda+np.dot(x.T,x))
    w = np.dot(np.dot(temp,x.T),y)
    return w

In [19]:
def RMS(x,y,w):
    x = pd.concat([pd.Series(1,index=x.index,name = 'w0'),x],axis=1) #將輸入資料補上bias項
    x = x.to_numpy() #轉換成矩陣來運算
    k = np.dot(x,w)
    p = k-y
    err = (p**2).iloc[:,0].sum()/(2*len(k))
    j = (err*2)**0.5
    return j

In [20]:
train_sigmoid = sigmoid_basisfunc(x_train)

In [30]:
train_sigmoid #已套入基底函數的500筆資料

,GRE_score,TOFEL_score,LOR,CGPA,Research
425,0.640595,0.651610,0.837279,0.893046,0.707964
91,0.175540,0.157652,0.504322,0.180163,0.244711
430,0.381201,0.371721,0.504322,0.323410,0.707964
411,0.423752,0.102563,0.104910,0.323410,0.244711
31,0.717497,0.334197,0.635889,0.387682,0.707964
...,...,...,...,...,...
298,0.768112,0.753876,0.749856,0.749723,0.707964
93,0.202654,0.157652,0.372154,0.240216,0.707964
442,0.783506,0.809727,0.749856,0.806560,0.707964
265,0.423752,0.298660,0.256686,0.542702,0.244711


In [31]:
def N_fold_rms(data,target,N,Lambda=0):#多設定了Lambda作為MAP參數使用
    RMS_list_train = []
    RMS_list_valid = []
    seg = int(len(data)/N) 
    for i in range(N): 
        if i == N-1: #避免資料數無法剛好整除N
            xvalid = data.iloc[i*seg:,:]
            xtrain = data.drop(index = xvalid.index)
            yvalid = target.iloc[i*seg:,:]
            ytrain = target.drop(index = yvalid.index)
        else:
            xvalid = data.iloc[i*seg:i*seg+seg,:]
            xtrain = data.drop(index = xvalid.index)
            yvalid = target.iloc[i*seg:i*seg+seg,:]
            ytrain = target.drop(index = yvalid.index)
        w_train = Weight(xtrain,ytrain,Lambda)
        res_train = RMS(xtrain,ytrain,w_train)
        res_valid = RMS(xvalid,yvalid,w_train)
        RMS_list_train.append(res_train)
        RMS_list_valid.append(res_valid)
    print("Lambda = {}的5-fold RMS\ntrain\n{}\n\n平均為:{}\n".format(Lambda,RMS_list_train,statistics.mean(RMS_list_train)))
    print("valid\n{}\n\n平均為:{}\n".format(RMS_list_valid,statistics.mean(RMS_list_valid)))
    return RMS_list_train,RMS_list_valid

In [29]:
lambda_list = [0,0.01,0.1,1,10,30]
for i in lambda_list:
  N_fold_rms(train_sigmoid,y_train,5,i)

Lambda = 0的5-fold RMS
train
[0.059828999316874826, 0.061695330066706126, 0.05964977832661263, 0.06236580421502309, 0.05926635415112024]

平均為:0.06056125321526738

valid
[0.06490819324157625, 0.0577676221872536, 0.06591242907352043, 0.05406594515507924, 0.06770384727058375]

平均為:0.062071607385602655

Lambda = 0.01的5-fold RMS
train
[0.05982904038688345, 0.06169536072831166, 0.0596498117295086, 0.06236584097969546, 0.05926640988010301]

平均為:0.060561292740900434

valid
[0.06490111469888883, 0.05777735525512537, 0.06591500451164162, 0.05406472647817993, 0.06768534529811117]

平均為:0.062068709248389385

Lambda = 0.1的5-fold RMS
train
[0.05983288417868764, 0.06169824058366792, 0.059652942343190345, 0.062369283079489406, 0.05927158881164662]

平均為:0.06056498779933638

valid
[0.0648418836875566, 0.0578651670425736, 0.06593910011960782, 0.05405743716841653, 0.0675308933691945]

平均為:0.06204689627746981

Lambda = 1的5-fold RMS
train
[0.06007153288780299, 0.06188191920123788, 0.05984951153170849, 0.06258